# # Final Submission for Introduction to Data Analytics for Business

My Final Submission is a study of tech salaries from multinational companies based in Ireland. The data is scraped from Levels.fyi. 
I will compare and contrast different career levels across different companies

# Import any Libraries required for the project

In [ ]:
#One time only 
#!pip install pandas-datareader
#!pip install matplotlib

In [ ]:
import pandas as pd 
import pandas_profiling as pp
import requests
import numpy as np 
from datetime import datetime 
import missingno as msno
import matplotlib.pyplot as plt 
import seaborn as sns 
import pandas_datareader as pdr
import warnings
import matplotlib

# Import Levels.fyi data and StockMarket Data

In [ ]:
data = requests.get('https://www.levels.fyi/js/salaryData.json').json()
df = pd.DataFrame(data)



In [ ]:
# Define the instruments to download. We would like to see Apple, Microsoft and Intel.
tickers = ['AAPL', 'MSFT', 'INTC','META']

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2022-01-01'
end_date = '2022-12-31'

# Use pandas_reader.data.DataReader to load the desired data. 
stock_data = pdr.data.DataReader(tickers, 'yahoo', start_date, end_date)
print(stock_data.head())
print(stock_data.dtypes)
print(stock_data.index)

# Inspect the levels.fyi dataframe

In [ ]:
#check if the data has imported ok using head()
df.head()

In [ ]:
#Check how many rows and columns are in the dataframe
df.shape

In [ ]:
#Check the datatypes
df.dtypes

In [ ]:
# Examine dataframe to examine columns and structure
df.info()

In [ ]:
#Pandas profiling takes several minutes to run so leaving commented out
#df.profile_report()


# Inspecting the Levels.fyi Data

Looking at the data values in the individual columns of my dataframe for anomolies

In [ ]:
#Check the values for years at company for any anomolies
print('Unique values for YearsAtCompany: ')
print(df['yearsatcompany'].unique())
#Can see people have entered in values of 90 
#Check how many rows have 90 for years at company
print((df['yearsatcompany']=='90').sum(), ' outlier with value 90 for yearsatcompany. This should be cleaned up later.')


In [ ]:
#Check the values for years of experience for any anomolies
print('Unique values for YearsOfExperience: ')
df['yearsofexperience'].unique()
#Can see people have entered in values of 90 - these rows should be dropped
#Check how many rows have 90 for years of experience
print((df['yearsofexperience']=='90').sum(), ' outlier with value 90 for Years Of Experience. This should be cleaned up later.')

In [ ]:
#Check the values for gender for any anomolies
print('Checking the Gender Column for anomalies: ')
print(df['gender'].unique())
#I can see nulls, Male, Female, Other, Title: Senior Software Engineer - this will need to be cleaned up also
print('Null values and Invalid rows will be cleaned up later. ')


In [ ]:
#Check the values for title for any anomolies
print('Checking Titles: ')
print(df['title'].unique())
#These look ok


In [ ]:
#dealing with NA's.  This function gives output in a single value if any null is present or not.
print('Checking for na values: ')
print (df.isna().sum())
# 0 in all column means there are no null values present in our dataset. If there was - I would have used the fillna() function to replace them with another value.

In [ ]:
#Check for Nulls
df.isnull().sum()
#All 0s mean there are none

In [ ]:
#Check for duplicates
print('Checking for duplicate rows: ')
print(df.duplicated().sum(), ' duplicate row(s) present.')


In [ ]:
# Check if rownumber is unique - I will use this as my index
df['rowNumber'].is_unique

# Cleaning the Data

I can see that a number of rows in my dataframe require cleaning: 

    I have extra columns that I dont require.  
    I have duplicate rows to clean
    My Columns are all being treated as Objects, but should be dates, floats, integars etc
    I have invalid entries for Gender


In [ ]:
# Drop unwanted columns
df.drop(columns=['level','tag','otherdetails','dmaid'], inplace=True)

#Check the dataframe again to confirm columns are gone
print(df.info())

In [ ]:
#Replace empty and invalid entries for gender with NotDisclosed
df['gender'].replace(['Title: Senior Software Engineer',''],'NotDisclosed', inplace=True)
print(df['gender'].unique())

In [ ]:
#Drop duplicate row if required
df.drop_duplicates (subset=None, keep='first', inplace=True)
#Check again for duplicates - they should be gone now
print(df.duplicated().sum(),' duplicate row(s) present.')

In [ ]:
# Convert some of the numeric columns that are classified as type 'object' to float and integar values
df[["totalyearlycompensation","basesalary","stockgrantvalue","bonus",]]=df[["totalyearlycompensation","basesalary","stockgrantvalue","bonus",]].astype(float)
df[["yearsofexperience","yearsatcompany","cityid"]]=df[["yearsofexperience","yearsatcompany","cityid"]].astype(int)
print (df.info())


In [ ]:
#Remove rows of data where the person has entered that they have over 50 years in the company
print('Maximum number of years in Company: ',df['yearsatcompany'].max())
print('Maximum number of years of Experience: ',df['yearsofexperience'].max())
print ('Cleaning up entries > 50 for YearsAtCompany and YearsOfExperience')
df=df[df['yearsatcompany']<51]
df=df[df['yearsofexperience']<51]
print('Maximum number of years in Company: ',df['yearsatcompany'].max())
print('Maximum number of years of Experience: ',df['yearsofexperience'].max())

In [ ]:
#Corinna Check accenture
# Are there any entries for Intel?
Intel=(df['company'] == 'Intel')
df.loc[Intel]
#957 entries

In [ ]:
#Convert all Company values to upper case to clean up examples such as amazon Amazon AMAZON
df['company']=df['company'].str.upper()
df['company']=df['company'].str.strip()


In [ ]:
#Convert timestamp to pandas datetime and keep just the date portion
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.date
#Add a new column for Date
#df['Date'] = df['timestamp'].dt.date
#df['Date_New'] = df['timestamp'].dt.date
#df.dtypes()

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
#Set the index on rownumber
#df.reset_index(inplace = True)
df.set_index('rowNumber',inplace=True)

In [ ]:
#Rename columns for readability
df.columns = ["Date","Company","Title","TCOMP","Location","YearsOfExperience","YearsOfService","BaseSalary","StockGrant","Bonus", "Gender", "CityId"]

In [ ]:
#check that the un-required columns are gone from the dataframe and that the numeric values are no longer showing as object types
df.info()

In [ ]:
#Maybe split the Location column?
#df[['City','Code']] = df['Location'].str.split(' ', expand=True)

In [ ]:
df.describe()

# Take a quick look at the survey results

In [ ]:
#Mean Number of years the person was in the company at the time of the survey
print("Mean Years of Service: " + str(df['YearsOfService'].mean()))

#Average Number of years of Experience the person had at the time of the survey
print("Mean Years of Experience: " + str(df['YearsOfExperience'].mean()))


In [ ]:
# Are there any entries for Intel?
Intel=(df['Company'] == 'INTEL')
df.loc[Intel]
#957 entries

In [ ]:
#Which Companies are located in Ireland
InIreland=df[df["Location"].str.contains("Ireland")]
print(InIreland['Company'].unique())
# Convert to List
FilteredCompanies=InIreland.Company.unique().tolist()

# Filter the dataframe for Multinational Companies of Interest based in Ireland

In [ ]:
print(FilteredCompanies)

In [ ]:
Filtereddf=(df['Company'].isin(FilteredCompanies)  & df['Location'].str.contains('Ireland',na=False))
CompaniesInIrelanddf=df.loc[Filtereddf]
CompaniesInIrelanddf

In [ ]:
CompaniesInIrelanddf.describe()

In [ ]:
#High Earners
High_Earners = (CompaniesInIrelanddf['TCOMP'] > 200)
CompaniesInIrelanddf.loc[High_Earners]

In [ ]:
#Top TCOMP Companies
IrishBaseSalarySorted=CompaniesInIrelanddf.sort_values(['TCOMP'],ascending=False)
IrishBaseSalarySorted.head()

In [ ]:

IrishBaseSalaryStatistics=IrishBaseSalarySorted.groupby('Company')['BaseSalary'].agg(['mean', 'min', 'max'])
IrishBaseSalaryStatistics.to_csv("IrishBaseSalaryStatistics1.csv")
print(IrishBaseSalaryStatistics)

In [ ]:
#company years of service statistics
IrishYearsOfServiceStatistics=CompaniesInIrelanddf.groupby('Company')['YearsOfService'].agg(['mean', 'min', 'max'])   
print(IrishYearsOfServiceStatistics)
#CompaniesInIrelanddf.to_csv('CorinnaCheckYearsOfService.csv')

In [ ]:
#export the CompaniesInIrelanddf dataframe to a csv file
CompaniesInIrelanddf.to_csv('CompaniesInIreland_SalaryData.csv')

In [ ]:
msno.matrix(CompaniesInIrelanddf, figsize=(10, 5))

In [ ]:
#CompaniesInIrelanddf.profile_report()

In [ ]:
#Filter Base Salary outliers - filter top and bottom 5% 
CompaniesInIrelanddf_filtered = CompaniesInIrelanddf[CompaniesInIrelanddf['BaseSalary'].between(df['BaseSalary'].quantile(.05),df['BaseSalary'].quantile(.95))]
CompaniesInIrelanddf_filtered = CompaniesInIrelanddf[CompaniesInIrelanddf['TCOMP'].between(df['TCOMP'].quantile(.05),df['BaseSalary'].quantile(.95))]

In [ ]:
CompaniesInIrelanddf_filtered.to_csv('CompaniesInIreland_SalaryData_filtered.csv')

# Visualize our Data

In [ ]:
#Visualise TCOMP for top 10 companies
TCOMPChart=CompaniesInIrelanddf_filtered.groupby(['Company'])['TCOMP'].mean().reset_index().sort_values('TCOMP',ascending = False )

# Set the figure size - handy for larger output
plt.rcParams["figure.figsize"] = [20, 10]
# Set up with a higher resolution screen
%config InlineBackend.figure_format = 'retina'

TCOMPChartTop10=TCOMPChart.nlargest(n=10, columns=['TCOMP'], keep="all")
#print(TCOMPChart.head(10))


font1 = {'family':'serif','color':'blue','size':25}
font2 = {'family':'serif','color':'darkred','size':20}

#Rotate the x-axis labels and define labels
plt.xticks(rotation=30, horizontalalignment="center")
plt.ylabel("TCOMP" , fontdict = font2)
plt.xlabel("Company" , fontdict = font2)
plt.title("TCOMP by Company" , fontdict = font1) 
plt.bar(TCOMPChartTop10["Company"],TCOMPChartTop10["TCOMP"])
plt.show()




In [ ]:
#Visualise TCOMP by Company and Gender using Seaborn
sns.set(rc = {'figure.figsize':(20,10)})

#Visualise TCOMP for top 10 companies
SalaryByGender=CompaniesInIrelanddf_filtered.groupby(['Company','Gender'])['BaseSalary'].mean().reset_index().sort_values('BaseSalary',ascending = False )
SalaryByGenderTop10=SalaryByGender.nlargest(n=10, columns=['BaseSalary'], keep="all")
# Set the figure size - handy for larger output
#plt.rcParams["figure.figsize"] = [20, 10]
# Set up with a higher resolution screen
#%config InlineBackend.figure_format = 'retina'

#SalaryByGenderTop10=SalaryByGender.nlargest(n=10, columns=['TCOMP'], keep="all")
print(SalaryByGenderTop10.head())
plt.xticks(rotation=-45)
sns.barplot(data=SalaryByGenderTop10, y='BaseSalary', x='Company', hue='Gender')





In [ ]:
#Create a Scattergraph Years of Experience to Total compensation

# Change seaborn plot size
fig = plt.gcf()
fig.set_size_inches(12, 8)

# Scatterplot arguments
#sns.set(rc = {'figure.figsize':(12,8)})
sns.lmplot(x='YearsOfExperience', y='TCOMP', data=CompaniesInIrelanddf_filtered,
           fit_reg=False, # No regression line
           hue='Gender').set(title='TCOMP per Year of Experience')   # Color by gender

plt.ylim(0, 300)
plt.xlim(-1, 35)

In [ ]:
# Suppress warnings
warnings.filterwarnings('ignore')

PairPlotdf=CompaniesInIrelanddf_filtered[["Company","TCOMP","BaseSalary","YearsOfService"]]
PairPlotdf.to_csv('PairPlot.csv')
data = pd.read_csv("PairPlot.csv")
sns.pairplot(data, hue='Company')

# Custom Function to Plot Years of Service versus TCOMP

In [ ]:
# Define a function called CorinnasPlotFunction - to fulfil scoring criteria

# Set the figure size - handy for larger output
plt.rcParams["figure.figsize"] = [20, 10]
# Set up with a higher resolution screen
#%config InlineBackend.figure_format = 'retina'
#matplotlib.style.use('fivethirtyeight')

#CompaniesInIrelanddf_filtered by Gender
FemaleSalaryIreland=(CompaniesInIrelanddf_filtered['Gender'].str.contains('Female',na=False))
FemaleSalaryIrelanddf=CompaniesInIrelanddf_filtered.loc[FemaleSalaryIreland]
FemaleSalaryIrelanddf=FemaleSalaryIrelanddf.groupby(['YearsOfExperience'])['TCOMP'].mean().reset_index().sort_values('YearsOfExperience',ascending = False )


MaleSalaryIreland=(CompaniesInIrelanddf_filtered['Gender'].str.contains('Male',na=False))
MaleSalaryIrelanddf=CompaniesInIrelanddf_filtered.loc[MaleSalaryIreland]
MaleSalaryIrelanddf=MaleSalaryIrelanddf.groupby(['YearsOfExperience'])['TCOMP'].mean().reset_index().sort_values('YearsOfExperience',ascending = False )


NDSalaryIreland=(CompaniesInIrelanddf_filtered['Gender'].str.contains('NotDisclosed',na=False))
NDSalaryIrelanddf=CompaniesInIrelanddf_filtered.loc[NDSalaryIreland]
NDSalaryIrelanddf=NDSalaryIrelanddf.groupby(['YearsOfExperience'])['TCOMP'].mean().reset_index().sort_values('YearsOfExperience',ascending = False )



def CorinnasPlotFunction(x, y,color,linestyle,linewidth,marker):
# Plot the inputs x,y in the provided colour
   ax.plot(x, y, color=color, linestyle=linestyle, linewidth=linewidth, marker=marker)

fig, ax = plt.subplots()
CorinnasPlotFunction(FemaleSalaryIrelanddf["YearsOfExperience"],FemaleSalaryIrelanddf["TCOMP"], "b","solid","8",'*')
CorinnasPlotFunction(MaleSalaryIrelanddf["YearsOfExperience"],MaleSalaryIrelanddf["TCOMP"], "g","dashed","7",'o')
CorinnasPlotFunction(NDSalaryIrelanddf["YearsOfExperience"],NDSalaryIrelanddf["TCOMP"], "r","dotted","6",'o')
ax.set_title("Years of Service versus TCOMP",fontdict = font1)

ax.set_xlabel("Years of Service", fontdict = font2)
ax.set_ylabel("TCOMP", fontdict = font2)
plt.show()


# Stock Data

In [ ]:
#Companies located in Ireland are in the InIreland dataframe

# Convert to List
FilteredTitles=InIreland.Title.unique().tolist()

#Filter out undesired titles such as Marketing
Irish_JobTitles_filtered = InIreland[InIreland['Title'].isin(['Software Engineer', 'Solution Architect', 'Business Analyst', 'Technical Program Manager', 'Data Scientist'])]
Irish_JobTitles_filtered.head()
newFilteredTitles=Irish_JobTitles_filtered.Title.unique().tolist()
print(newFilteredTitles)


Irish_StockGrantCompanies_filtered = Irish_JobTitles_filtered[Irish_JobTitles_filtered['StockGrant']>0]
#Irish_StockGrantCompanies_filtered.to_csv("Irish_StockGrantCompanies_filtered1.csv")


newFilteredCompanies=Irish_StockGrantCompanies_filtered.Company.unique().tolist()
print(newFilteredCompanies)

#Drop Companies from the dataframe where unable to find a ticker in yahoo finance'
NewSalarydf = Irish_StockGrantCompanies_filtered[~Irish_StockGrantCompanies_filtered['Company'].isin(['INTERCOM', 'STARTUP', 'SLACK', 'STRIPE', 'AWS', 'WALMART LABS', 'QUANTCAST','LINKEDIN','RIOT GAMES', 'FIREEYE', 'HUAWEI', 'REDDIT'])]

# Stock ticker dictionary
StockTicker = {'AMAZON':'AMZN', 'MICROSOFT':'MSFT','CISCO':'CSCO', 'GOOGLE':'GOOG', 'WORKDAY':'WDAY', 'FACEBOOK':'META','TOAST':'TOST', 'ARISTA NETWORKS':'ANET','MONGODB':'MDB','HUBSPOT':'HUBS','PIVOTAL':'PICC','TWILIO':'TWLO','EXPEDIA':'EXPE','SHUTTERSTOCK':'SSTK','VMWARE':'VMW','TENABLE':'TENB','QUALTRICS':'XM','SERVICENOW':'NOW','RAPID7':'RPD','ZALANDO':'ZLNDY','ZENDESK':'ZEN'}
  
# Mapping the dictionary keys to the data frame.
NewSalarydf['Ticker'] = NewSalarydf['Company'].map(StockTicker)
NewSalarydf.to_csv("NewSalarydf2.csv")


In [ ]:
# Define the instruments to download. We would like to see Apple, Microsoft and Intel.

tickers = ['XM','NOW','RPD','ZLNDY','ZEN','AMZN','CSCO','MSFT','GOOG','META','AAPL','MSFT', 'WDAY','GWRE','INTC','ORCL','TOST','ANET','MDB','HUBS','PICC','TWLO','EXPE','SSTK','VMW','TENB']
start_date = '2022-11-11'
end_date = '2022-11-11'

# Use pandas_reader.data.DataReader to load the desired data. 
stock_data = pdr.data.DataReader(tickers, 'yahoo', start_date, end_date)
print(stock_data.head())
stock_data.to_csv("StockData.csv")
#print(stock_data.dtypes)
#print(stock_data.index)


In [ ]:
stock_data=stock_data.transpose()

#Find the Stock Opening Price


In [ ]:
stock_data.head(50)
#stock_data.info()
#stock_data=stock_data[stock_data['Attributes'].isin(['Open'])]
#stock_data.to_csv("TransposedStockData1.csv")

In [ ]:
#stock_data.to_csv("TransposedStock.csv")
stock_data = pd.read_csv('TransposedStock.csv')
stock_data = stock_data[stock_data['Attributes'].isin(['Open'])]
stock_data.set_index('Symbols')
stock_data.columns = ["RowNum","Attributes","Ticker","OpeningPrice"]
stock_data.head()

In [ ]:
stock_data.info()

# Merge Salary Dataframe and StockPrice Dataframe

In [ ]:
#Test Merge
MergeCompanyStockdf = pd.merge(NewSalarydf,
                 stock_data[['Ticker', 'OpeningPrice']],
                 on='Ticker')

MergeCompanyStockdf.head()


In [ ]:
MergeCompanyStockdf.to_csv("MergeCompanyStock.csv")

In [ ]:
Corinnas Project 
References 
    https://towardsdatascience.com/a-beginners-guide-to-grabbing-and-analyzing-salary-data-in-python-e8c60eab186e
    #https://www.datasciencelearner.com/yahoo-finance-api-python/
   https://www.youtube.com/watch?v=sgndYho8RyI
    https://www.analyticsvidhya.com/blog/2021/12/stock-market-analysis-with-pandas-datareader-and-plotly-for-beginners/
        https://www.shanelynn.ie/bar-plots-in-python-using-pandas-dataframes/
        